In [4]:
!pip install topmost pyyaml gensim ipywidgets widgetsnbextension pandas-profiling

     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 60.4/60.4 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/84.4 kB ? eta -:--:--
     ---------------------------------------- 84.4/84.4 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/162.3 kB ? eta -:--:--
     --------------------------------- ---- 143.4/162.3 kB 4.3 MB/s eta 0:00:01
     -------------------------------------- 162.3/162.3 kB 3.2 MB/s eta 0:00:00
  Using cached pyparsing-3.1.1-py3-none-any.whl.metadata (5.1 kB)
   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   ---------------------------------------- 139.4/139.4 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   --- ----

In [5]:
!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run script troubleshoot

Jupyter command `jupyter-nbextension` not found.


In [6]:
import topmost

import gensim
from ECRTMhandler import DataHandler, Preprocessings

import pandas as pd

In [7]:
# 複数GPUがある場合は数値で選択可能
device = "cuda" # or "cpu"

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_train

,Title,Excerpt,Category
0,"Uefa Opens Proceedings against Barcelona, Juve...",Uefa has opened disciplinary proceedings again...,sports
1,Amazon Blames Inflation as It Increases Cost o...,The increases are steeper than the 17 percent ...,business
2,Nigeria’s Parliament Passes Amended Electoral ...,Nigeria's Senate on Tuesday passed the harmoni...,politics
3,Nigeria: Lagos Governor Tests Positive for Cov...,"The Lagos State Governor, Mr. Babajide Sanwo-O...",health
4,South Africa Calls For Calm as Electoral Refor...,South Africa has raised concerns about the det...,politics
...,...,...,...
4681,Uganda Drops Charges Against Nigerian Singers ...,Nigerian singers Omah Lay and Tems are set to ...,entertainment
4682,NNPC Cuts Directors’ Fees by 60.2%,The Nigerian National Petroleum Corporation (N...,business
4683,Nigeria’s External Reserves Hit $37bn with $3....,Nigeria’s foreign reserves will get a timely b...,business
4684,Transcorp Hilton Wins Africa’s Leading Busines...,Transcorp Hilton Abuja has been honoured with ...,business


In [3]:
# preprocess raw data
stop_words_4_topmost = list(gensim.parsing.preprocessing.STOPWORDS)
preprocessing = Preprocessings(stopwords=stop_words_4_topmost)
parsed_texts, bow_matrix, vocab, word_embeddings = preprocessing.parse(texts=data)
print(parsed_texts)

Found training documents 3


===>parse texts: 100%|██████████| 3/3 [00:00<?, ?it/s]

Real vocab size: 4
===>convert to matrix...



c:\Users\iceplanet\miniconda3\envs\ECRTMcustom\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[==================================================] 100.0% 252.1/252.1MB downloaded


===>making word embeddings: 100%|██████████| 4/4 [00:00<00:00, 636.01it/s]

===> number of found embeddings: 3/4
['hello world', 'sentense writing', 'hello']


In [ ]:
########################### Neural Topic Models ####################################
# dataset for neural topic models
dataset = DataHandler(data, device=device, read_labels=True, as_tensor=True)

# create a model
model = topmost.models.ECRTM(vocab_size=dataset.vocab_size, pretrained_WE=dataset.pretrained_WE)
model = model.to(device)

# create a trainer
trainer = topmost.trainers.BasicTrainer(model, dataset)

# train the model
trainer.train()